<h1> Générer notre Access Token Sellsy </h1>

In [ ]:
import requests
import base64
import json
import pandas as pd
from pandas import json_normalize


client_id = "renseigne ici ton id"
client_secret = "renseigne ici ton secret"

#Encoder en base64
client_creds = f"{client_id}:{client_secret}"
client_creds_b64 = base64.b64encode(client_creds.encode()).decode()

token_url = " https://login.sellsy.com/oauth2/access-tokens"
headers = {
    "Authorization": f"Basic {client_creds_b64}",
}

data = {
    "grant_type": "client_credentials"
}

response = requests.post(token_url, headers=headers, data=data)

if response.status_code == 200:
    access_token = response.json().get("access_token")
    print("Token obtenu :", access_token)
else:
    print("Erreur lors de l'obtention du token :", response.status_code, response.text)

<h1>Récupérer le contenu de <i> Invoices </i> au format JSON </h1>

In [ ]:
if access_token:
    api_url_invoices = "https://api.sellsy.com/v2/invoices"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    all_data_invoices = []
    pagination = []
    count = 0
    total = 0
    offset = 0

    while True:

        # Construction de l'URL avec le décalage
        url = f"{api_url_invoices}?offset={offset}"

        # Effectuer la requête HTTP
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            break

        # Charger les données JSON
        data = response.json()

        # Ajouter les résultats à la liste
        all_data_invoices.extend(data['data'])
        count += data['pagination'].get('count', None)
        total = data['pagination'].get('total', None)
        print(f"count = {count} et total = {total}")

        # Vérifier s'il existe une page suivante
        if count < total :
            offset += 100  # ou une autre logique selon la structure de votre API
        else:
            break

    print(all_data_invoices)

<h1> Transformer notre JSON en un dataframe Pandas puis Spark pour la création d'une table dans notre Lakehouse </h1>

In [ ]:
extracted_invoices = []

for entry in all_data_invoices:
        id_ = entry.get('id', None)
        number = entry.get('number', None)
        status = entry.get('status', None)
        date = entry.get('date', [None])
        due_date = entry.get('due_date', [None])
        amount = entry.get('amount', {}).get('total_excl_tax', None)
        company_name = entry.get('_embed', {}).get('company', {}).get('name', None)


        extracted_invoices.append({
            'id': id_,
            'number': number,
            'status': status,
            'date': date,
            'due_date': due_date,
            'amount': amount,
            'company_name' : company_name
        })

df_invoices = pd.DataFrame(extracted_invoices)

display(df_invoices)

#Transformation du pandas DF to Spark DF
sparkDF=spark.createDataFrame(df_invoices)

<h1> Ecrire dans notre Lakehouse </h1>

In [ ]:
#Ecrire le DataFrame Spark en Table dans le lakehouse
sparkDF.write.format("delta").mode("overwrite").saveAsTable("FactInvoices")